This notebook compares the time it takes to solve the system of ODEs for the Lotka-Volterra problem with and without the sensitivity equations included.

The original system includes 2 ODEs, the sensitivity model included 8 additional equations.

In [1]:
import time
start_time = time.time()

In [2]:
import logging

import numpy as np

from scipy.integrate import solve_ivp

import lotka_volterra
import utils.caching
from utils.caching import make_cached_s3
from utils.paths import S3_BUCKET_NAME

In [3]:
logging.basicConfig()
logging.getLogger(utils.caching.__name__).setLevel(logging.DEBUG)

Directory where results of expensive calculations will be stored:

In [4]:
cached = make_cached_s3(S3_BUCKET_NAME)

Read-only input datasets:

In [5]:
@cached(item_type=np.ndarray, batch_size=lotka_volterra.n_chains, read_only=True)
def rw_samples() -> list[np.ndarray]:
    raise NotImplementedError

In [6]:
def solve_without_sensitivity(theta):
    return solve_ivp(lotka_volterra.lotka_volterra, lotka_volterra.t_span, lotka_volterra.u_init, args=(theta,), dense_output=True)

In [7]:
uw_init = np.concatenate([np.array(lotka_volterra.u_init), np.zeros(lotka_volterra.d * lotka_volterra.q)])

def solve_with_sensitivity(theta):
    return solve_ivp(lotka_volterra.lotka_volterra_sensitivity, lotka_volterra.t_span, uw_init, args=(theta,), dense_output=True)

Make sure that the sample is loaded from the cache:

In [8]:
samples = rw_samples[0]

DEBUG:utils.caching:Reading from disk cache: rw_samples_0
DEBUG:utils.caching:Reading from S3 gradient-free-mcmc-postprocessing/rw_samples_0.npy


In [9]:
n = 10_000

Record the calculation time for the ODE without the sensitivity equations:

In [10]:
%%time
np.apply_along_axis(solve_without_sensitivity, 1, samples[:n]);

CPU times: user 27.9 s, sys: 120 ms, total: 28 s
Wall time: 28 s


Record the calculation time for the ODE with the sensitivity equations:

In [11]:
%%time
np.apply_along_axis(solve_with_sensitivity, 1, samples[:n]);

CPU times: user 42.1 s, sys: 120 ms, total: 42.2 s
Wall time: 42.2 s


Notebook execution took:

In [12]:
time.time() - start_time

74.77855610847473